In [2]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
import datetime
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

import re

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import StaleElementReferenceException


# Dentro do while, após abrir o site:
#wait = WebDriverWait(driver, 10)


data_de_hoje = datetime.date.today()

#service = Service(executable_path="/Users/franciscoabraao/Documents/PROJPESCA 2021/JUBART/Acompanhamento_de_Precos/chromedriver")
carrefour = "https://mercado.carrefour.com.br/s?q=frango&sort=score_desc&page=0"
j = 1
precos_carrefour = []

### COLETA DOS DADOS

In [3]:
j = 43

In [144]:
j = 43
precos_carrefour = []
while j < 44: 
    service = ChromeService()
    driver = webdriver.Chrome(service=service)
    driver.get(carrefour)

    time.sleep(4)
    driver.find_element(By.CSS_SELECTOR, r"#__next > div:nth-child(5) > div.bg-white.min-h-\[39px\].border-b.border-gray-light > div > div > div").click()
    time.sleep(2)
    driver.find_element(By.XPATH, "/html/body/div[4]/div[2]/div[2]/div/div/div[2]/div[2]/div/section/div/div[1]/button[2]").click()
    time.sleep(4)
    select = Select(driver.find_element(By.CSS_SELECTOR, "#selectCity"))

    if select.options[j].is_enabled():
        select.select_by_index(j)
    else: 
        j += 1
        driver.quit()
        continue

    time.sleep(4)
    botao_cidade = driver.find_elements(By.CSS_SELECTOR, "div[class='grid grid-flow-col']")
    total_lojas = len(botao_cidade)
    print(f"[INFO] Cidade {j} possui {total_lojas} loja(s)")

    for i in range(total_lojas - 1):
        try:
            # ⬇️ Rola gradativamente até a loja i
            scroll_container = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div[class*='overflow-y-scroll']"))
            )
            for _ in range(i):
                driver.execute_script("arguments[0].scrollBy(0, 40)", scroll_container)
                time.sleep(2)

            botao_cidade = driver.find_elements(By.CSS_SELECTOR, "div[class='grid grid-flow-col']")
            loja, _, cidade, _ = botao_cidade[i].text.split("\n")
            botao_cidade[i].click()

            print(f"[INFO] Acessando loja {loja} na cidade {cidade}")


            time.sleep(5)
            for _ in range(10):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            divs_carrefour = driver.find_elements(By.CSS_SELECTOR, "section[data-product-card-content='true']")
            for div in divs_carrefour:
                dtxt = div.text.strip()
                if dtxt:
                    precos_carrefour.append(f"{dtxt} | {loja} | {cidade}")

            print(f"[INFO] {loja} - {cidade} - {len(divs_carrefour)} produtos encontrados")

            driver.back()
            time.sleep(4)

            # Reabrir site e cidade
            driver.get(carrefour)
            time.sleep(4)
            driver.find_element(By.CSS_SELECTOR, r"#__next > div:nth-child(5) > div.bg-white.min-h-\[39px\].border-b.border-gray-light > div > div > div").click()
            time.sleep(2)
            driver.find_element(By.XPATH, "/html/body/div[4]/div[2]/div[2]/div/div/div[2]/div[2]/div/section/div/div[1]/button[2]").click()
            time.sleep(4)

            select = Select(driver.find_element(By.CSS_SELECTOR, "#selectCity"))
            if select.options[j].is_enabled():
                select.select_by_index(j)
            else: 
                j += 1
                driver.quit()
                continue

            time.sleep(4)

        except Exception as e:
            print(f"[ERRO] Loja {i} da cidade {j}: {e}")
            continue

    j += 1
    driver.quit()


[INFO] Cidade 43 possui 6 loja(s)
[INFO] Acessando loja Hipermercado Campinas Valinhos na cidade Jardim Von Zuben, Campinas - SP
[INFO] Hipermercado Campinas Valinhos - Jardim Von Zuben, Campinas - SP - 20 produtos encontrados
[INFO] Acessando loja Carrefour Bairro Campinas Cambui na cidade Cambui, Campinas - SP
[INFO] Carrefour Bairro Campinas Cambui - Cambui, Campinas - SP - 20 produtos encontrados
[ERRO] Loja 2 da cidade 43: Message: element click intercepted: Element <div class="grid grid-flow-col">...</div> is not clickable at point (282, 746). Other element would receive the click: <a aria-label="... visite este link para saber mais sobre cookies. Abre em uma nova janela" tabindex="0" class="cc-link" href="https://www.carrefour.com.br/institucional/politicas-carrefour/politica-de-privacidade-carrefour" rel="noopener noreferrer nofollow" target="_blank" title="Clique aqui para saber mais. Este link abre em uma nova janela">Acesse nossa Política de Privacidade.</a>
  (Session info:

In [145]:

dados_brutos = precos_carrefour

dados_estruturados = []

for linha in dados_brutos:
    try:
        produto_bloco, loja, endereco = linha.split(" | ")
        linhas = produto_bloco.strip().split("\n")

        # Heurística com base no número de linhas
        if len(linhas) >= 4:
            produto_nome = next((x for x in linhas if "R$" not in x and "%" not in x and "promoção" not in x), "").strip()
            preco_final = linhas[-1].strip() if "R$" in linhas[-1] else None
            desconto = next((x for x in linhas if "%" in x and "-" in x), None)
        else:
            produto_nome = next((x for x in linhas if "R$" not in x), "").strip()
            preco_final = next((x for x in linhas if "R$" in x), None)
            desconto = None

        cidade_estado = endereco.split(",")[-1].strip()
        cidade, estado = cidade_estado.split(" - ")

        preco_final = float(preco_final.replace("R$", "").replace(",", ".").strip()) if preco_final else None
        desconto = float(desc.replace("%", "").replace(",", ".").strip()) if (desc := desconto) else None

        dados_estruturados.append({
            "produto": produto_nome,
            "preco": preco_final,
            "desconto": desconto,
            "loja": loja.strip(),
            "endereco": endereco.strip(),
            "cidade": cidade.strip(),
            "estado": estado.strip()
        })

    except Exception as e:
        print(f"Erro ao processar linha: {linha}\n-> {e}")

df_produtos = pd.DataFrame(dados_estruturados)

# import ace_tools as tools; tools.display_dataframe_to_user(name="Produtos Carrefour Estruturados", dataframe=df_produtos)
df_produtos 


,produto,preco,desconto,loja,endereco,cidade,estado
0,Filé de Peito de Frango Congelado sem Osso Per...,20.90,-5.0,Hipermercado Campinas Valinhos,"Jardim Von Zuben, Campinas - SP",Campinas,SP
1,Filé Sassami de Peito de Frango Congelado Sadi...,23.55,-11.0,Hipermercado Campinas Valinhos,"Jardim Von Zuben, Campinas - SP",Campinas,SP
2,3 Unidades de Filé de Peito de Frango Congelad...,62.70,NaN,Hipermercado Campinas Valinhos,"Jardim Von Zuben, Campinas - SP",Campinas,SP
3,Filé de Frango Sadia Assa Fácil 700 g,20.79,NaN,Hipermercado Campinas Valinhos,"Jardim Von Zuben, Campinas - SP",Campinas,SP
4,Filé de Peito de Frango Congelado Swift 1 Kg,25.65,NaN,Hipermercado Campinas Valinhos,"Jardim Von Zuben, Campinas - SP",Campinas,SP
5,Filezinho Sassami Empanado Swift 700 g,24.90,NaN,Hipermercado Campinas Valinhos,"Jardim Von Zuben, Campinas - SP",Campinas,SP
6,Filé Sassami de Frango Congelado Swift 1 Kg,23.90,NaN,Hipermercado Campinas Valinhos,"Jardim Von Zuben, Campinas - SP",Campinas,SP
7,Filé de Frango à parmegiana Sadia 500g,22.65,NaN,Hipermercado Campinas Valinhos,"Jardim Von Zuben, Campinas - SP",Campinas,SP
8,Filé de Peito de Frango Congelado Swift do Cam...,27.99,NaN,Hipermercado Campinas Valinhos,"Jardim Von Zuben, Campinas - SP",Campinas,SP
9,Filé de Sobrecoxa de Frango Swift 1Kg,22.90,NaN,Hipermercado Campinas Valinhos,"Jardim Von Zuben, Campinas - SP",Campinas,SP


In [146]:
df_produtos['loja'].unique()

array(['Hipermercado Campinas Valinhos',
       'Carrefour Bairro Campinas Cambui',
       'Hipermercado Campinas Shopping Outlet'], dtype=object)

In [83]:
import pandas as pd

# Lista de dados brutos (trecho representativo da variável 'precos_carrefour')
dados_brutos = precos_carrefour

dados_estruturados = []

for linha in dados_brutos:
    try:
        produto_bloco, loja, endereco = linha.split(" | ")
        linhas = produto_bloco.strip().split("\n")

        # Heurística com base no número de linhas
        if len(linhas) >= 4:
            # Promoção com texto, preço original, desconto e preço final
            produto_nome = next((x for x in linhas if "R$" not in x and "%" not in x and "promoção" not in x), "").strip()
            preco_final = linhas[-1].strip() if "R$" in linhas[-1] else None
            desconto = next((x for x in linhas if "%" in x and "-" in x), None)
        else:
            # Sem promoção
            produto_nome = next((x for x in linhas if "R$" not in x), "").strip()
            preco_final = next((x for x in linhas if "R$" in x), None)
            desconto = None

        cidade_estado = endereco.split(",")[-1].strip()
        cidade, estado = cidade_estado.split(" - ")

        # Processar valores
        preco_final = float(preco_final.replace("R$", "").replace(",", ".").strip()) if preco else None
        desconto = float(desconto.replace("%", "").replace(",", ".").strip()) if (desc := desconto) else None

        dados_estruturados.append({
            "produto": produto_nome,
            "preco": preco_final,
            "desconto": desconto,
            "loja": loja.strip(),
            "endereco": endereco.strip(),
            "cidade": cidade.strip(),
            "estado": estado.strip()
        })

    except Exception as e:
        print(f"Erro ao processar linha: {linha}\n-> {e}")

df_produtos = pd.DataFrame(dados_estruturados)


# import ace_tools as tools; tools.display_dataframe_to_user(name="Produtos Carrefour Estruturados", dataframe=df_produtos)

df_produtos


Erro ao processar linha: Filé de Peito de Frango Congelado sem Osso Perdigão 1Kg - Sassami
R$ 21,99
-5%
R$ 20,90 | Hipermercado Tamboré | Centro Empresarial Tamboré, Barueri - SP
-> name 'preco' is not defined
Erro ao processar linha: 20% OFF
Badge da promoção 20% OFF
Filezinho Sassami Seara Orgânico IQF 600g
R$ 35,59
-20%
R$ 28,47 | Hipermercado Tamboré | Centro Empresarial Tamboré, Barueri - SP
-> name 'preco' is not defined
Erro ao processar linha: Filé Sassami de Peito de Frango Congelado Sadia 1Kg
R$ 23,29 | Hipermercado Tamboré | Centro Empresarial Tamboré, Barueri - SP
-> name 'preco' is not defined
Erro ao processar linha: 3 Unidades de Filé de Peito de Frango Congelado sem Osso Perdigão 1Kg - Sassami
R$ 62,70 | Hipermercado Tamboré | Centro Empresarial Tamboré, Barueri - SP
-> name 'preco' is not defined
Erro ao processar linha: 20% OFF
Badge da promoção 20% OFF
Filezinho de Peito Sassami DaGranja IQF 1Kg
R$ 32,99
-20%
R$ 26,39 | Hipermercado Tamboré | Centro Empresarial Tambo

""


In [80]:
df_produtos

""


In [63]:
precos_carrefour2 = precos_carrefour.copy()

In [64]:
while j < 20: ### MUDAR AQUI PARA ESCOLHAR A QUANTIDADE DE CIDADES
    # Definir a versão específica do ChromeDriver
    
    # service = ChromeService(ChromeDriverManager(driver_version="129.0.6668.90").install())
    service = ChromeService()

    # Iniciar o driver
    driver = webdriver.Chrome(service=service)
    #driver  = webdriver.Chrome(service=service)
    driver.get(carrefour)

    time.sleep(4)

    driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(5) > div.bg-white.min-h-\[39px\].border-b.border-gray-light > div > div > div").click()
    time.sleep(2)
    driver.find_element(By.XPATH, "/html/body/div[4]/div[2]/div[2]/div/div/div[2]/div[2]/div/section/div/div[1]/button[2]").click()
    #botao = driver.find_element(By.CLASS_NAME, "border.border-gray-300.items-center.flex.flex-col.text-center.rounded-lg.shadow-md.hover:ring-1.ring-slate-300.pointer-events-auto.select-none.cursor-pointer.p-2.w-1/2.sm:w-1/3.h-40.pt-[21px].max-w-[128px].overflow-hidden")
    #botao.click()

    time.sleep(4)
    #select = Select(driver.find_element(By.CLASS_NAME, "searchPickupPointsSelect"))

    select = Select(driver.find_element(By.CSS_SELECTOR, "#selectCity"))

    
    if select.options[j].is_enabled():
        select.select_by_index(j)
    else: 
        j += 1
        driver.quit()
        continue
    
    time.sleep(4)
    botao_cidade = driver.find_elements(By.CSS_SELECTOR, "div[class='grid grid-flow-col']")
    loja, _, cidade, _ = botao_cidade[0].text.split("\n")
    botao_cidade[0].click()
    
    time.sleep(5)
    for i in range(10):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    divs_carrefour = driver.find_elements(By.CSS_SELECTOR, "section[data-product-card-content='true']")

    for div in divs_carrefour:
        dtxt = div.text
        if len(dtxt) > 0:
            precos_carrefour.append(dtxt + " " + loja + "" + cidade)
    
    j += 1

    driver.quit()


<>:14: SyntaxWarning: invalid escape sequence '\['
<>:14: SyntaxWarning: invalid escape sequence '\['
/var/folders/hn/ff5wd5md5hz9v1n8vb5bf9f80000gn/T/ipykernel_47597/509768736.py:14: SyntaxWarning: invalid escape sequence '\['
  driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(5) > div.bg-white.min-h-\[39px\].border-b.border-gray-light > div > div > div").click()


In [65]:
j

20

In [66]:
precos_carrefour_20 = precos_carrefour.copy()


In [ ]:
precos_carrefour_20

In [ ]:
while j < 40: ### MUDAR AQUI PARA ESCOLHAR A QUANTIDADE DE CIDADES
    # Definir a versão específica do ChromeDriver
    
    # service = ChromeService(ChromeDriverManager(driver_version="129.0.6668.90").install())
    service = ChromeService()

    # Iniciar o driver
    driver = webdriver.Chrome(service=service)
    #driver  = webdriver.Chrome(service=service)
    driver.get(carrefour)

    time.sleep(4)

    driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(5) > div.bg-white.min-h-\[39px\].border-b.border-gray-light > div > div > div").click()
    time.sleep(2)
    driver.find_element(By.XPATH, "/html/body/div[4]/div[2]/div[2]/div/div/div[2]/div[2]/div/section/div/div[1]/button[2]").click()
    #botao = driver.find_element(By.CLASS_NAME, "border.border-gray-300.items-center.flex.flex-col.text-center.rounded-lg.shadow-md.hover:ring-1.ring-slate-300.pointer-events-auto.select-none.cursor-pointer.p-2.w-1/2.sm:w-1/3.h-40.pt-[21px].max-w-[128px].overflow-hidden")
    #botao.click()

    time.sleep(4)
    #select = Select(driver.find_element(By.CLASS_NAME, "searchPickupPointsSelect"))

    select = Select(driver.find_element(By.CSS_SELECTOR, "#selectCity"))

    
    if select.options[j].is_enabled():
        select.select_by_index(j)
    else: 
        j += 1
        driver.quit()
        continue
    
    time.sleep(4)
    botao_cidade = driver.find_elements(By.CSS_SELECTOR, "div[class='grid grid-flow-col']")
    loja, _, cidade, _ = botao_cidade[0].text.split("\n")
    botao_cidade[0].click()
    
    time.sleep(5)
    for i in range(10):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    divs_carrefour = driver.find_elements(By.CSS_SELECTOR, "section[data-product-card-content='true']")

    for div in divs_carrefour:
        dtxt = div.text
        if len(dtxt) > 0:
            precos_carrefour.append(dtxt + " " + loja + "" + cidade)
    
    j += 1

    driver.quit()


In [73]:
j

40

In [ ]:
precos_carrefour

In [75]:
precos_carrefour_40 = precos_carrefour.copy()

In [ ]:
while j < 73: ### MUDAR AQUI PARA ESCOLHAR A QUANTIDADE DE CIDADES
    # Definir a versão específica do ChromeDriver
    
    # service = ChromeService(ChromeDriverManager(driver_version="129.0.6668.90").install())
    service = ChromeService()

    # Iniciar o driver
    driver = webdriver.Chrome(service=service)
    #driver  = webdriver.Chrome(service=service)
    driver.get(carrefour)

    time.sleep(4)

    driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(5) > div.bg-white.min-h-\[39px\].border-b.border-gray-light > div > div > div").click()
    time.sleep(2)
    driver.find_element(By.XPATH, "/html/body/div[4]/div[2]/div[2]/div/div/div[2]/div[2]/div/section/div/div[1]/button[2]").click()
    #botao = driver.find_element(By.CLASS_NAME, "border.border-gray-300.items-center.flex.flex-col.text-center.rounded-lg.shadow-md.hover:ring-1.ring-slate-300.pointer-events-auto.select-none.cursor-pointer.p-2.w-1/2.sm:w-1/3.h-40.pt-[21px].max-w-[128px].overflow-hidden")
    #botao.click()

    time.sleep(4)
    #select = Select(driver.find_element(By.CLASS_NAME, "searchPickupPointsSelect"))

    select = Select(driver.find_element(By.CSS_SELECTOR, "#selectCity"))

    
    if select.options[j].is_enabled():
        select.select_by_index(j)
    else: 
        j += 1
        driver.quit()
        continue
    
    time.sleep(4)
    botao_cidade = driver.find_elements(By.CSS_SELECTOR, "div[class='grid grid-flow-col']")
    loja, _, cidade, _ = botao_cidade[0].text.split("\n")
    botao_cidade[0].click()
    
    time.sleep(5)
    for i in range(10):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    divs_carrefour = driver.find_elements(By.CSS_SELECTOR, "section[data-product-card-content='true']")

    for div in divs_carrefour:
        dtxt = div.text
        if len(dtxt) > 0:
            precos_carrefour.append(dtxt + " " + loja + "" + cidade)
    
    j += 1

    driver.quit()


In [81]:
j=71

In [ ]:
precos_carrefour

In [ ]:
precos_carrefour1 = pd.DataFrame(precos_carrefour)
precos_carrefour1['data'] = data_de_hoje
precos_carrefour1['data'] = precos_carrefour1['data'].apply(lambda x: x.strftime('%d/%m/%Y'))
cols = list(precos_carrefour1)
cols.insert(0, cols.pop(cols.index('data')))
precos_carrefour1 = precos_carrefour1.loc[:, cols]
precos_carrefour1

In [ ]:
precos_carrefour1.to_excel('precos_carrefour_kani_20250516.xlsx') 

# FIM DA COLETA DE DADOS

# TRATAMENTO DOS DADOS

In [2]:
precos_carrefour = pd.read_excel('precos_carrefour_kani_20250516_tratado.xlsx') 

In [3]:
precos_carrefour


,Unnamed: 0.1,Unnamed: 0,data,produto,preco,desconto,loja,cidade,uf,lat,long
0,0,0,17/05/2025,Kani Congelado Frescatto 200g,13.55,NaN,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL,-9.64959,-35.71739
1,1,1,17/05/2025,Kani Congelado Frescatto 200g,12.99,NaN,Big Bompreço ItabunaGoes Calmon,Itabuna - BA,BA,-14.79790,-39.27272
2,2,2,17/05/2025,Kani Congelado Frescatto 200g,12.69,NaN,Hipermercado Brasilia SulZona Industrial (Guará),Brasília - DF,DF,-15.82938,-47.95478
3,3,3,17/05/2025,Kani Congelado Frescatto 200g,12.19,NaN,Hipermercado Brasilia TaguatingaAguas Claras,Taguatinga - DF,DF,-15.82804,-48.04138
4,4,4,17/05/2025,Kani Congelado Costa Sul 200g,13.55,NaN,Hipermercado Vila Velha ShoppingDivino Espirit...,Vila Velha - ES,ES,-20.34885,-40.29805
...,...,...,...,...,...,...,...,...,...,...,...
63,63,65,17/05/2025,Kani Congelado Costa Sul 200g,14.79,NaN,Carrefour Bairro São Carlos - Sta FelíciaSanta...,São Carlos - SP,SP,-21.99732,-47.91609
64,64,66,17/05/2025,Kani Congelado Costa Sul 200g,10.29,NaN,Hipermercado São José Dos CamposSerimbura,São José dos Campos - SP,SP,-23.22279,-45.90544
65,65,67,17/05/2025,Kani Congelado Costa Sul 200g,9.89,NaN,Hipermercado Rio Preto ShoppingJd Fernandes,São Jose Rio Preto - SP,SP,-20.83233,-49.39859
66,66,68,17/05/2025,Kani Congelado Frescatto 200g,9.98,NaN,Hipermercado Rio Preto ShoppingJd Fernandes,São Jose Rio Preto - SP,SP,-20.83233,-49.39859


In [22]:
### Quebramos a coluna 0
df_quebra_produto_col_0 = precos_carrefour[0].str.split('\n', expand=True)

In [ ]:
df_quebra_produto_col_0

In [ ]:
df_quebra_produto_col_0.info()

#### CASO DO KANI que a tabela só tem 3 colunas

In [25]:
df_quebra_produto_col_0[df_quebra_produto_col_0[2].notnull()]

# Tratar os dados da coluna 5: onde a coluna 5 não é nula, mover para a coluna 1 e limpar a coluna 5
mask_percent = df_quebra_produto_col_0[2].notnull()
df_quebra_produto_col_0.loc[mask_percent, 1] = df_quebra_produto_col_0.loc[mask_percent, 3]
df_quebra_produto_col_0.loc[mask_percent, 3] = None

df_quebra_produto_col_0

,0,1,2,3
0,Kani Congelado Frescatto 200g,"R$ 13,55 Hiper Maceió JatiúcaMangabeiras, Mace...",None,None
1,Kani Congelado Frescatto 200g,"R$ 12,99 Big Bompreço ItabunaGoes Calmon, Itab...",None,None
2,Kani Congelado Frescatto 200g,"R$ 12,69 Hipermercado Brasilia SulZona Industr...",None,None
3,Kani Congelado Frescatto 200g,"R$ 12,19 Hipermercado Brasilia TaguatingaAguas...",None,None
4,Kani Congelado Costa Sul 200g,"R$ 13,55 Hipermercado Vila Velha ShoppingDivin...",None,None
...,...,...,...,...
69,Kani Congelado Costa Sul 200g,"R$ 14,79 Carrefour Bairro MorumbiJardim Londri...",None,None
70,Kani Congelado Costa Sul 200g,"R$ 9,89 Hipermercado São VicenteJd Independenc...",None,None
71,Kani Congelado Costa Sul 200g,"R$ 10,29 Hipermercado Sorocaba NorteVila Carva...",None,None
72,Kani Congelado Costa Sul 200g,"R$ 10,49 Carrefour Bairro Taubaté IICentro, Ta...",None,None


In [26]:
precos_carrefour1 = df_quebra_produto_col_0[[0, 1, 2]]
precos_carrefour1.columns = ['produto', 'preco_loja', 'desconto']


#import re

# Extrair o primeiro preço encontrado (R$ xx,xx)
precos_carrefour1['preco'] = precos_carrefour1['preco_loja'].str.extract(r'R\$ ?([\d.,]+)')
precos_carrefour1['preco'] = precos_carrefour1['preco'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)

# Extrair nome da loja (após o preço, até a próxima vírgula)
precos_carrefour1['loja'] = precos_carrefour1['preco_loja'].str.extract(r'R\$ ?[\d.,]+\s*([^\d\n,]+)')

# Separar tudo o que estiver depois da segunda vírgula na coluna 'sobra'
precos_carrefour1['cidade'] = precos_carrefour1['preco_loja'].str.split(',', n=2).str[2]

# Extrair as duas letras após o hífen na coluna 'cidade' para criar a coluna 'uf'
precos_carrefour1['uf'] = precos_carrefour1['cidade'].str.extract(r'-\s*([A-Z]{2})')

precos_carrefour1 = precos_carrefour1[['produto', 'preco', 'desconto', 'loja', 'cidade', 'uf']]

precos_carrefour1

,produto,preco,desconto,loja,cidade,uf
0,Kani Congelado Frescatto 200g,13.55,None,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL
1,Kani Congelado Frescatto 200g,12.99,None,Big Bompreço ItabunaGoes Calmon,Itabuna - BA,BA
2,Kani Congelado Frescatto 200g,12.69,None,Hipermercado Brasilia SulZona Industrial (Guará),Brasília - DF,DF
3,Kani Congelado Frescatto 200g,12.19,None,Hipermercado Brasilia TaguatingaAguas Claras,Taguatinga - DF,DF
4,Kani Congelado Costa Sul 200g,13.55,None,Hipermercado Vila Velha ShoppingDivino Espirit...,Vila Velha - ES,ES
...,...,...,...,...,...,...
69,Kani Congelado Costa Sul 200g,14.79,None,Carrefour Bairro MorumbiJardim Londrina,São Paulo - SP,SP
70,Kani Congelado Costa Sul 200g,9.89,None,Hipermercado São VicenteJd Independencia,São Vicente - SP,SP
71,Kani Congelado Costa Sul 200g,10.29,None,Hipermercado Sorocaba NorteVila Carvalho,Sorocaba/Votorantim - SP,SP
72,Kani Congelado Costa Sul 200g,10.49,None,Carrefour Bairro Taubaté IICentro,Taubate - SP,SP


In [27]:
precos_carrefour1['data'] = data_de_hoje
precos_carrefour1['data'] = precos_carrefour1['data'].apply(lambda x: x.strftime('%d/%m/%Y'))
cols = list(precos_carrefour1)
cols.insert(0, cols.pop(cols.index('data')))
precos_carrefour1 = precos_carrefour1.loc[:, cols]
precos_carrefour1

,data,produto,preco,desconto,loja,cidade,uf
0,17/05/2025,Kani Congelado Frescatto 200g,13.55,None,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL
1,17/05/2025,Kani Congelado Frescatto 200g,12.99,None,Big Bompreço ItabunaGoes Calmon,Itabuna - BA,BA
2,17/05/2025,Kani Congelado Frescatto 200g,12.69,None,Hipermercado Brasilia SulZona Industrial (Guará),Brasília - DF,DF
3,17/05/2025,Kani Congelado Frescatto 200g,12.19,None,Hipermercado Brasilia TaguatingaAguas Claras,Taguatinga - DF,DF
4,17/05/2025,Kani Congelado Costa Sul 200g,13.55,None,Hipermercado Vila Velha ShoppingDivino Espirit...,Vila Velha - ES,ES
...,...,...,...,...,...,...,...
69,17/05/2025,Kani Congelado Costa Sul 200g,14.79,None,Carrefour Bairro MorumbiJardim Londrina,São Paulo - SP,SP
70,17/05/2025,Kani Congelado Costa Sul 200g,9.89,None,Hipermercado São VicenteJd Independencia,São Vicente - SP,SP
71,17/05/2025,Kani Congelado Costa Sul 200g,10.29,None,Hipermercado Sorocaba NorteVila Carvalho,Sorocaba/Votorantim - SP,SP
72,17/05/2025,Kani Congelado Costa Sul 200g,10.49,None,Carrefour Bairro Taubaté IICentro,Taubate - SP,SP


In [28]:
precos_carrefour1.to_excel('precos_carrefour_kani_20250516_tratado.xlsx') 

### CAso dos empanados que a coluna tem 5 colunas

In [ ]:
### Quebramos a coluna 0
df_quebra_produto_col_0 = precos_carrefour[0].str.split('\n', expand=True)

# Selecionar apenas as linhas do df_split onde apenas as colunas 0 e 1 têm dados (colunas 2, 3, 4, 5 são todas nulas)
df_final = df_quebra_produto_col_0[df_quebra_produto_col_0[[2, 3, 4, 5]].isnull().all(axis=1)][[0, 1]].copy()


# Subtrair os índices de df_apenas_2colunas de df_split para obter apenas as linhas "sobrando"
df_sobra = df_quebra_produto_col_0.drop(df_final.index)


### TRATAR os dados da couna 0 e 1 com "OFF" alterando os dados de lugar
mask_off_0 = df_sobra[0].str.contains("OFF", na=False)
df_sobra.loc[mask_off_0, 0] = df_sobra.loc[mask_off_0, 2]
df_sobra.loc[mask_off_0, 2] = None

mask_off_1 = df_sobra[1].str.contains("OFF", na=False)
df_sobra.loc[mask_off_1, 1] = df_sobra.loc[mask_off_1, 3]
df_sobra.loc[mask_off_1, 3] = None

### Tratar os dados da coluna 2 - que será o desconto 
mask_percent = df_sobra[2].astype(str).str.contains("%", na=False)
df_sobra.loc[mask_percent, 1] = df_sobra.loc[mask_percent, 3]
df_sobra.loc[mask_percent, 3] = None

### Tratar os dados da coluna 4 - que será o desconto 
mask_percent = df_sobra[4].astype(str).str.contains("%", na=False)
df_sobra.loc[mask_percent, 2] = df_sobra.loc[mask_percent, 4]
df_sobra.loc[mask_percent, 4] = None

# Tratar os dados da coluna 5: onde a coluna 5 não é nula, mover para a coluna 1 e limpar a coluna 5
mask_percent = df_sobra[5].notnull()
df_sobra.loc[mask_percent, 1] = df_sobra.loc[mask_percent, 5]
df_sobra.loc[mask_percent, 5] = None

# Eliminar linhas em que a coluna 3 não é nula
df_sobra= df_sobra[df_sobra[3].isnull()]

###### JUNTANDO OS DADOS
precos_carrefour1 = pd.concat([df_sobra, df_final], axis=0)
precos_carrefour1 = precos_carrefour1.reset_index(drop=True)
precos_carrefour1 = precos_carrefour1.rename(columns={0: 'produto', 1: 'preco_loja', 2: 'desconto'})

precos_carrefour1 = precos_carrefour1[['produto', 'preco_loja', 'desconto']]


#import re

# Extrair o primeiro preço encontrado (R$ xx,xx)
precos_carrefour1['preco'] = precos_carrefour1['preco_loja'].str.extract(r'R\$ ?([\d.,]+)')
precos_carrefour1['preco'] = precos_carrefour1['preco'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)

# Extrair nome da loja (após o preço, até a próxima vírgula)
precos_carrefour1['loja'] = precos_carrefour1['preco_loja'].str.extract(r'R\$ ?[\d.,]+\s*([^\d\n,]+)')

# Separar tudo o que estiver depois da segunda vírgula na coluna 'sobra'
precos_carrefour1['cidade'] = precos_carrefour1['preco_loja'].str.split(',', n=2).str[2]

# Extrair as duas letras após o hífen na coluna 'cidade' para criar a coluna 'uf'
precos_carrefour1['uf'] = precos_carrefour1['cidade'].str.extract(r'-\s*([A-Z]{2})')

precos_carrefour1 = precos_carrefour1[['produto', 'preco', 'desconto', 'loja', 'cidade', 'uf']]

precos_carrefour1['data'] = data_de_hoje
precos_carrefour1['data'] = precos_carrefour1['data'].apply(lambda x: x.strftime('%d/%m/%Y'))
cols = list(precos_carrefour1)
cols.insert(0, cols.pop(cols.index('data')))
precos_carrefour1 = precos_carrefour1.loc[:, cols]


precos_carrefour1




,produto,preco,desconto,loja,cidade,uf
0,Empanado Chicken Crispy Tradicional Seara 300g,12.49,None,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL
1,Iscas De Frango Tradicional Empanado Crocante ...,14.65,None,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL
2,Empanado de Frango Steak Recheio Presunto de P...,4.09,None,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL
3,Empanado De Frango Incrível 100% Vegetal 300g,17.85,-43%,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL
4,Empanado de Frango Vegetal Congelado Incrível!...,28.39,None,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL
...,...,...,...,...,...,...
523,Quadritos de Peixe Empanado Costa Sul Pacote 400g,9.98,NaN,Hipermercado Sorocaba NorteVila Carvalho,Sorocaba/Votorantim - SP,SP
524,Steak de Peixe Empanado Costa Sul Pacote 100g,3.99,NaN,Hipermercado Sorocaba NorteVila Carvalho,Sorocaba/Votorantim - SP,SP
525,Steak de Peixe Empanado Costa Sul Pacote 100g,3.99,NaN,Hipermercado TaboãoCentro,Taboão da Serra - SP,SP
526,Steak de Peixe Empanado Costa Sul Pacote 100g,3.99,NaN,Carrefour Bairro Taubaté IICentro,Taubate - SP,SP


In [4]:
precos_carrefour1

NameError: name 'precos_carrefour1' is not defined

### LOCALIZAR LOJA

In [5]:
precos_carrefour1 = pd.read_excel('precos_carrefour_kani_20250516_tratado.xlsx') 

In [ ]:
# Ler a planilha do Excel com as informações das lojas e preços
df_lojas = pd.read_excel('lista_lojas_final.xlsx')

# Mesclar as duas planilhas com base no nome da loja
precos_carrefour1 = pd.merge(precos_carrefour1, df_lojas[['loja', 'lat', 'long']], on=['loja'])

precos_carrefour1['lat'].isnull().any()

precos_carrefour1 

In [10]:
# Mapeamento de estados para regiões
uf_to_region = {
    'AC': 'Norte', 'AP': 'Norte', 'AM': 'Norte', 'PA': 'Norte', 'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
    'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste', 'PB': 'Nordeste', 'PE': 'Nordeste',
    'PI': 'Nordeste', 'RN': 'Nordeste', 'SE': 'Nordeste',
    'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
    'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul'
}

# Criar a nova coluna com base no mapeamento
precos_carrefour1 ['regiao'] = precos_carrefour1 ['uf'].map(uf_to_region)

precos_carrefour1 

,Unnamed: 0.1,data,Unnamed: 0,produto,preco,desconto,loja,cidade,uf,lat,long,regiao
0,0,17/05/2025,0,Empanado Chicken Crispy Tradicional Seara 300g,12.49,NaN,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL,-9.64959,-35.71739,Nordeste
1,1,17/05/2025,1,Iscas De Frango Tradicional Empanado Crocante ...,14.65,NaN,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL,-9.64959,-35.71739,Nordeste
2,2,17/05/2025,2,Empanado de Frango Steak Recheio Presunto de P...,4.09,NaN,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL,-9.64959,-35.71739,Nordeste
3,3,17/05/2025,3,Empanado De Frango Incrível 100% Vegetal 300g,17.85,-43%,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL,-9.64959,-35.71739,Nordeste
4,4,17/05/2025,4,Empanado de Frango Vegetal Congelado Incrível!...,28.39,NaN,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL,-9.64959,-35.71739,Nordeste
...,...,...,...,...,...,...,...,...,...,...,...,...
459,459,17/05/2025,459,Empanado de Frango Tradicional Crocante Swift ...,10.90,NaN,Carrefour Bairro MorumbiJardim Londrina,São Paulo - SP,SP,-23.61172,-46.74075,Sudeste
460,460,17/05/2025,460,Empanado de Frango com Queijo Crocante Swift 3...,10.90,NaN,Carrefour Bairro MorumbiJardim Londrina,São Paulo - SP,SP,-23.61172,-46.74075,Sudeste
461,461,17/05/2025,461,Atum Sólido em Óleo Gomes da Costa 170g,12.99,NaN,Carrefour Bairro MorumbiJardim Londrina,São Paulo - SP,SP,-23.61172,-46.74075,Sudeste
462,462,17/05/2025,462,Atum Sólido ao Natural Gomes da Costa 170g,12.99,NaN,Carrefour Bairro MorumbiJardim Londrina,São Paulo - SP,SP,-23.61172,-46.74075,Sudeste


In [11]:
precos_carrefour1.to_excel('precos_carrefour_empanado_20250516_tratado.xlsx') 

In [9]:
precos_carrefour1 = pd.read_excel('precos_carrefour_empanado_20250516_tratado.xlsx') 

In [ ]:

precos_carrefour1['data'] = data_de_hoje
precos_carrefour1['data'] = precos_carrefour1['data'].apply(lambda x: x.strftime('%d/%m/%Y'))
cols = list(precos_carrefour1)
cols.insert(0, cols.pop(cols.index('data')))
precos_carrefour1 = precos_carrefour1.loc[:, cols]


precos_carrefour1

### Ajuste no csv de kani

In [ ]:
precos_carrefour1 = pd.read_csv('precos_carrefour_kani.csv') 
precos_carrefour1 


In [ ]:
precos_carrefour1['data'] = data_de_hoje
precos_carrefour1['data'] = precos_carrefour1['data'].apply(lambda x: x.strftime('%d/%m/%Y'))
cols = list(precos_carrefour1)
cols.insert(0, cols.pop(cols.index('data')))
precos_carrefour1 = precos_carrefour1.loc[:, cols]
precos_carrefour1 

In [52]:
precos_carrefour1['bairro'] = precos_carrefour1['cidade_estado'].str.split(',', n=1).str[0]
precos_carrefour1['cidade_estado'] = precos_carrefour1['cidade_estado'].str.split(',', n=1).str[1].str.strip()
precos_carrefour1

,data,produto,preco,loja,cidade_estado,bairro
0,18/05/2025,Kani Congelado Frescatto 200g,"R$ 13,39",Hiper Maceió Jatiúca,Maceió - AL,Mangabeiras
1,18/05/2025,Kani Congelado Frescatto 200g,"R$ 12,99",Big Bompreço Itabuna,Itabuna - BA,Goes Calmon
2,18/05/2025,Kani Congelado Frescatto 200g,"R$ 12,59",Hipermercado Brasilia Sul,Brasília - DF,Zona Industrial (Guará)
3,18/05/2025,Kani Congelado Frescatto 200g,"R$ 12,19",Hipermercado Brasilia Taguatinga,Taguatinga - DF,Aguas Claras
4,18/05/2025,Kani Congelado Costa Sul 200g,"R$ 13,39",Hipermercado Vila Velha Shopping,Vila Velha - ES,Divino Espirito Santo
...,...,...,...,...,...,...
91,18/05/2025,Kani Congelado Costa Sul 200g,"R$ 10,19",Hipermercado Sorocaba Norte,Sorocaba/Votorantim - SP,Vila Carvalho
92,18/05/2025,Kani Congelado Costa Sul 200g,"R$ 10,19",Hipermercado Taboão,Taboão da Serra - SP,Centro
93,18/05/2025,Kani Congelado Costa Sul 200g,"R$ 10,49",Carrefour Bairro Taubaté II,Taubate - SP,Centro
94,18/05/2025,Kani Congelado Costa Sul 200g,"R$ 14,79",Carrefour Bairro Vinhedo,Vinhedo - SP,Vista Alegre


In [53]:
precos_carrefour1['loja_bairro'] = precos_carrefour1['loja'] + precos_carrefour1['bairro']
precos_carrefour1

,data,produto,preco,loja,cidade_estado,bairro,loja_bairro
0,18/05/2025,Kani Congelado Frescatto 200g,"R$ 13,39",Hiper Maceió Jatiúca,Maceió - AL,Mangabeiras,Hiper Maceió JatiúcaMangabeiras
1,18/05/2025,Kani Congelado Frescatto 200g,"R$ 12,99",Big Bompreço Itabuna,Itabuna - BA,Goes Calmon,Big Bompreço ItabunaGoes Calmon
2,18/05/2025,Kani Congelado Frescatto 200g,"R$ 12,59",Hipermercado Brasilia Sul,Brasília - DF,Zona Industrial (Guará),Hipermercado Brasilia SulZona Industrial (Guará)
3,18/05/2025,Kani Congelado Frescatto 200g,"R$ 12,19",Hipermercado Brasilia Taguatinga,Taguatinga - DF,Aguas Claras,Hipermercado Brasilia TaguatingaAguas Claras
4,18/05/2025,Kani Congelado Costa Sul 200g,"R$ 13,39",Hipermercado Vila Velha Shopping,Vila Velha - ES,Divino Espirito Santo,Hipermercado Vila Velha ShoppingDivino Espirit...
...,...,...,...,...,...,...,...
91,18/05/2025,Kani Congelado Costa Sul 200g,"R$ 10,19",Hipermercado Sorocaba Norte,Sorocaba/Votorantim - SP,Vila Carvalho,Hipermercado Sorocaba NorteVila Carvalho
92,18/05/2025,Kani Congelado Costa Sul 200g,"R$ 10,19",Hipermercado Taboão,Taboão da Serra - SP,Centro,Hipermercado TaboãoCentro
93,18/05/2025,Kani Congelado Costa Sul 200g,"R$ 10,49",Carrefour Bairro Taubaté II,Taubate - SP,Centro,Carrefour Bairro Taubaté IICentro
94,18/05/2025,Kani Congelado Costa Sul 200g,"R$ 14,79",Carrefour Bairro Vinhedo,Vinhedo - SP,Vista Alegre,Carrefour Bairro VinhedoVista Alegre


In [54]:
precos_carrefour1 = precos_carrefour1.rename(columns={'loja': 'loja_2', 'loja_bairro': 'loja'})
precos_carrefour1

,data,produto,preco,loja_2,cidade_estado,bairro,loja
0,18/05/2025,Kani Congelado Frescatto 200g,"R$ 13,39",Hiper Maceió Jatiúca,Maceió - AL,Mangabeiras,Hiper Maceió JatiúcaMangabeiras
1,18/05/2025,Kani Congelado Frescatto 200g,"R$ 12,99",Big Bompreço Itabuna,Itabuna - BA,Goes Calmon,Big Bompreço ItabunaGoes Calmon
2,18/05/2025,Kani Congelado Frescatto 200g,"R$ 12,59",Hipermercado Brasilia Sul,Brasília - DF,Zona Industrial (Guará),Hipermercado Brasilia SulZona Industrial (Guará)
3,18/05/2025,Kani Congelado Frescatto 200g,"R$ 12,19",Hipermercado Brasilia Taguatinga,Taguatinga - DF,Aguas Claras,Hipermercado Brasilia TaguatingaAguas Claras
4,18/05/2025,Kani Congelado Costa Sul 200g,"R$ 13,39",Hipermercado Vila Velha Shopping,Vila Velha - ES,Divino Espirito Santo,Hipermercado Vila Velha ShoppingDivino Espirit...
...,...,...,...,...,...,...,...
91,18/05/2025,Kani Congelado Costa Sul 200g,"R$ 10,19",Hipermercado Sorocaba Norte,Sorocaba/Votorantim - SP,Vila Carvalho,Hipermercado Sorocaba NorteVila Carvalho
92,18/05/2025,Kani Congelado Costa Sul 200g,"R$ 10,19",Hipermercado Taboão,Taboão da Serra - SP,Centro,Hipermercado TaboãoCentro
93,18/05/2025,Kani Congelado Costa Sul 200g,"R$ 10,49",Carrefour Bairro Taubaté II,Taubate - SP,Centro,Carrefour Bairro Taubaté IICentro
94,18/05/2025,Kani Congelado Costa Sul 200g,"R$ 14,79",Carrefour Bairro Vinhedo,Vinhedo - SP,Vista Alegre,Carrefour Bairro VinhedoVista Alegre


In [55]:
# Ler a planilha do Excel com as informações das lojas e preços
df_lojas = pd.read_excel('lista_lojas_final.xlsx')
df_lojas

,Unnamed: 0,uf,cidade,loja,lat,long
0,0,AL,Maceió/AL,Hiper Maceió JatiúcaMangabeiras,-9.64959,-35.71739
1,1,AM,Manaus/AM,Hipermercado Manaus AdrianópolisAdrianopolis,-3.10513,-60.01123
2,2,AM,Manaus/AM,Hipermercado Manaus ShoppingChapada,-3.09326,-60.02191
3,3,AM,Manaus/AM,Hipermercado Ponta NegraDom Pedro,-3.09195,-60.04980
4,4,BA,Itabuna/BA,Big Bompreço ItabunaGoes Calmon,-14.79790,-39.27272
...,...,...,...,...,...,...
112,112,SP,São Paulo/SP,Hipermercado InterlagosCampo Grande,-23.67753,-46.67525
113,113,SP,São Paulo/SP,Hipermercado LimãoLimao,-23.51228,-46.67499
114,114,SP,São Paulo/SP,Hipermercado PamplonaJd Pauilsta,-23.57010,-46.66136
115,115,SP,São Paulo/SP,Hipermercado PinheirosVila Gomes,-23.62417,-46.71079


In [ ]:
# Mesclar as duas planilhas com base no nome da loja
precos_carrefour1 = pd.merge(precos_carrefour1, df_lojas[['loja', 'lat', 'long', 'cidade', 'uf']], on=['loja'])

#precos_carrefour1['lat'].isnull().any()

precos_carrefour1 

In [57]:
precos_carrefour1.head(2)

,data,produto,preco,loja_2,cidade_estado,bairro,loja,lat,long,cidade,uf
0,18/05/2025,Kani Congelado Frescatto 200g,"R$ 13,39",Hiper Maceió Jatiúca,Maceió - AL,Mangabeiras,Hiper Maceió JatiúcaMangabeiras,-9.64959,-35.71739,Maceió/AL,AL
1,18/05/2025,Kani Congelado Frescatto 200g,"R$ 12,99",Big Bompreço Itabuna,Itabuna - BA,Goes Calmon,Big Bompreço ItabunaGoes Calmon,-14.79790,-39.27272,Itabuna/BA,BA


In [ ]:
precos_carrefour1 = precos_carrefour1.rename(columns={'cidade': 'cidade2', 'cidade_estado': 'cidade'})
precos_carrefour1

In [62]:
# Extrair o primeiro preço encontrado (R$ xx,xx)
precos_carrefour1['preco'] = precos_carrefour1['preco'].str.extract(r'R\$ ?([\d.,]+)')
precos_carrefour1['preco'] = precos_carrefour1['preco'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)


# Extrair as duas letras após o hífen na coluna 'cidade' para criar a coluna 'uf'
precos_carrefour1['uf'] = precos_carrefour1['cidade'].str.extract(r'-\s*([A-Z]{2})')

precos_carrefour1

,data,produto,preco,loja_2,cidade,bairro,loja,lat,long,cidade2,uf
0,18/05/2025,Kani Congelado Frescatto 200g,13.39,Hiper Maceió Jatiúca,Maceió - AL,Mangabeiras,Hiper Maceió JatiúcaMangabeiras,-9.64959,-35.71739,Maceió/AL,AL
1,18/05/2025,Kani Congelado Frescatto 200g,12.99,Big Bompreço Itabuna,Itabuna - BA,Goes Calmon,Big Bompreço ItabunaGoes Calmon,-14.79790,-39.27272,Itabuna/BA,BA
2,18/05/2025,Kani Congelado Frescatto 200g,12.59,Hipermercado Brasilia Sul,Brasília - DF,Zona Industrial (Guará),Hipermercado Brasilia SulZona Industrial (Guará),-15.82938,-47.95478,Brasília/DF,DF
3,18/05/2025,Kani Congelado Frescatto 200g,12.19,Hipermercado Brasilia Taguatinga,Taguatinga - DF,Aguas Claras,Hipermercado Brasilia TaguatingaAguas Claras,-15.82804,-48.04138,Taguatinga/DF,DF
4,18/05/2025,Kani Congelado Costa Sul 200g,13.39,Hipermercado Vila Velha Shopping,Vila Velha - ES,Divino Espirito Santo,Hipermercado Vila Velha ShoppingDivino Espirit...,-20.34885,-40.29805,Vila Velha/ES,ES
...,...,...,...,...,...,...,...,...,...,...,...
85,18/05/2025,Kani Congelado Costa Sul 200g,10.19,Hipermercado São José Dos Campos,São José dos Campos - SP,Serimbura,Hipermercado São José Dos CamposSerimbura,-23.22279,-45.90544,São José dos Campos/SP,SP
86,18/05/2025,Hossomaki Philadelphia Kani Kama 160 g,27.49,Hipermercado São José Dos Campos,São José dos Campos - SP,Serimbura,Hipermercado São José Dos CamposSerimbura,-23.22279,-45.90544,São José dos Campos/SP,SP
87,18/05/2025,Kani Congelado Costa Sul 200g,9.89,Hipermercado Rio Preto Shopping,São Jose Rio Preto - SP,Jd Fernandes,Hipermercado Rio Preto ShoppingJd Fernandes,-20.83233,-49.39859,São Jose Rio Preto/SP,SP
88,18/05/2025,Kani Congelado Frescatto 200g,9.98,Hipermercado Rio Preto Shopping,São Jose Rio Preto - SP,Jd Fernandes,Hipermercado Rio Preto ShoppingJd Fernandes,-20.83233,-49.39859,São Jose Rio Preto/SP,SP


In [64]:
# Mapeamento de estados para regiões
uf_to_region = {
    'AC': 'Norte', 'AP': 'Norte', 'AM': 'Norte', 'PA': 'Norte', 'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
    'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste', 'PB': 'Nordeste', 'PE': 'Nordeste',
    'PI': 'Nordeste', 'RN': 'Nordeste', 'SE': 'Nordeste',
    'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
    'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul'
}

# Criar a nova coluna com base no mapeamento
precos_carrefour1 ['regiao'] = precos_carrefour1 ['uf'].map(uf_to_region)

precos_carrefour1 

,data,produto,preco,loja_2,cidade,bairro,loja,lat,long,cidade2,uf,regiao
0,18/05/2025,Kani Congelado Frescatto 200g,13.39,Hiper Maceió Jatiúca,Maceió - AL,Mangabeiras,Hiper Maceió JatiúcaMangabeiras,-9.64959,-35.71739,Maceió/AL,AL,Nordeste
1,18/05/2025,Kani Congelado Frescatto 200g,12.99,Big Bompreço Itabuna,Itabuna - BA,Goes Calmon,Big Bompreço ItabunaGoes Calmon,-14.79790,-39.27272,Itabuna/BA,BA,Nordeste
2,18/05/2025,Kani Congelado Frescatto 200g,12.59,Hipermercado Brasilia Sul,Brasília - DF,Zona Industrial (Guará),Hipermercado Brasilia SulZona Industrial (Guará),-15.82938,-47.95478,Brasília/DF,DF,Centro-Oeste
3,18/05/2025,Kani Congelado Frescatto 200g,12.19,Hipermercado Brasilia Taguatinga,Taguatinga - DF,Aguas Claras,Hipermercado Brasilia TaguatingaAguas Claras,-15.82804,-48.04138,Taguatinga/DF,DF,Centro-Oeste
4,18/05/2025,Kani Congelado Costa Sul 200g,13.39,Hipermercado Vila Velha Shopping,Vila Velha - ES,Divino Espirito Santo,Hipermercado Vila Velha ShoppingDivino Espirit...,-20.34885,-40.29805,Vila Velha/ES,ES,Sudeste
...,...,...,...,...,...,...,...,...,...,...,...,...
85,18/05/2025,Kani Congelado Costa Sul 200g,10.19,Hipermercado São José Dos Campos,São José dos Campos - SP,Serimbura,Hipermercado São José Dos CamposSerimbura,-23.22279,-45.90544,São José dos Campos/SP,SP,Sudeste
86,18/05/2025,Hossomaki Philadelphia Kani Kama 160 g,27.49,Hipermercado São José Dos Campos,São José dos Campos - SP,Serimbura,Hipermercado São José Dos CamposSerimbura,-23.22279,-45.90544,São José dos Campos/SP,SP,Sudeste
87,18/05/2025,Kani Congelado Costa Sul 200g,9.89,Hipermercado Rio Preto Shopping,São Jose Rio Preto - SP,Jd Fernandes,Hipermercado Rio Preto ShoppingJd Fernandes,-20.83233,-49.39859,São Jose Rio Preto/SP,SP,Sudeste
88,18/05/2025,Kani Congelado Frescatto 200g,9.98,Hipermercado Rio Preto Shopping,São Jose Rio Preto - SP,Jd Fernandes,Hipermercado Rio Preto ShoppingJd Fernandes,-20.83233,-49.39859,São Jose Rio Preto/SP,SP,Sudeste


In [66]:
precos_carrefour1 = precos_carrefour1[['data','produto', 'preco', 'loja', 'cidade', 'uf', 'lat', 'long', 'regiao']]
precos_carrefour1




,data,produto,preco,loja,cidade,uf,lat,long,regiao
0,18/05/2025,Kani Congelado Frescatto 200g,13.39,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL,-9.64959,-35.71739,Nordeste
1,18/05/2025,Kani Congelado Frescatto 200g,12.99,Big Bompreço ItabunaGoes Calmon,Itabuna - BA,BA,-14.79790,-39.27272,Nordeste
2,18/05/2025,Kani Congelado Frescatto 200g,12.59,Hipermercado Brasilia SulZona Industrial (Guará),Brasília - DF,DF,-15.82938,-47.95478,Centro-Oeste
3,18/05/2025,Kani Congelado Frescatto 200g,12.19,Hipermercado Brasilia TaguatingaAguas Claras,Taguatinga - DF,DF,-15.82804,-48.04138,Centro-Oeste
4,18/05/2025,Kani Congelado Costa Sul 200g,13.39,Hipermercado Vila Velha ShoppingDivino Espirit...,Vila Velha - ES,ES,-20.34885,-40.29805,Sudeste
...,...,...,...,...,...,...,...,...,...
85,18/05/2025,Kani Congelado Costa Sul 200g,10.19,Hipermercado São José Dos CamposSerimbura,São José dos Campos - SP,SP,-23.22279,-45.90544,Sudeste
86,18/05/2025,Hossomaki Philadelphia Kani Kama 160 g,27.49,Hipermercado São José Dos CamposSerimbura,São José dos Campos - SP,SP,-23.22279,-45.90544,Sudeste
87,18/05/2025,Kani Congelado Costa Sul 200g,9.89,Hipermercado Rio Preto ShoppingJd Fernandes,São Jose Rio Preto - SP,SP,-20.83233,-49.39859,Sudeste
88,18/05/2025,Kani Congelado Frescatto 200g,9.98,Hipermercado Rio Preto ShoppingJd Fernandes,São Jose Rio Preto - SP,SP,-20.83233,-49.39859,Sudeste


In [67]:
precos_carrefour1.to_excel('precos_carrefour_kani_20250518_tratado.xlsx') 

In [ ]:
precos_carrefour2 = pd.read_excel('precos_carrefour_empanado_20250516_tratado.xlsx') 
precos_carrefour2

In [37]:
precos_carrefour1.columns.unique()

Index(['data', 'produto', 'preco', 'loja_2', 'cidade_estado', 'bairro', 'loja',
       'lat', 'long', 'cidade', 'uf'],
      dtype='object')

In [ ]:
precos_carrefour1 = precos_carrefour1['data', 'produto', 'preco', 'loja','loja_2', 'cidade_estado', 'bairro', 
       'lat', 'long', 'cidade', 'uf']

In [46]:
precos_carrefour2 = pd.read_excel('precos_carrefour_empanado_20250516_tratado.xlsx') 
precos_carrefour2


,Unnamed: 0.2,Unnamed: 0.1,data,Unnamed: 0,produto,preco,desconto,loja,cidade,uf,lat,long,regiao
0,0,0,17/05/2025,0,Empanado Chicken Crispy Tradicional Seara 300g,12.49,NaN,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL,-9.64959,-35.71739,Nordeste
1,1,1,17/05/2025,1,Iscas De Frango Tradicional Empanado Crocante ...,14.65,NaN,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL,-9.64959,-35.71739,Nordeste
2,2,2,17/05/2025,2,Empanado de Frango Steak Recheio Presunto de P...,4.09,NaN,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL,-9.64959,-35.71739,Nordeste
3,3,3,17/05/2025,3,Empanado De Frango Incrível 100% Vegetal 300g,17.85,-43%,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL,-9.64959,-35.71739,Nordeste
4,4,4,17/05/2025,4,Empanado de Frango Vegetal Congelado Incrível!...,28.39,NaN,Hiper Maceió JatiúcaMangabeiras,Maceió - AL,AL,-9.64959,-35.71739,Nordeste
...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,459,459,17/05/2025,459,Empanado de Frango Tradicional Crocante Swift ...,10.90,NaN,Carrefour Bairro MorumbiJardim Londrina,São Paulo - SP,SP,-23.61172,-46.74075,Sudeste
460,460,460,17/05/2025,460,Empanado de Frango com Queijo Crocante Swift 3...,10.90,NaN,Carrefour Bairro MorumbiJardim Londrina,São Paulo - SP,SP,-23.61172,-46.74075,Sudeste
461,461,461,17/05/2025,461,Atum Sólido em Óleo Gomes da Costa 170g,12.99,NaN,Carrefour Bairro MorumbiJardim Londrina,São Paulo - SP,SP,-23.61172,-46.74075,Sudeste
462,462,462,17/05/2025,462,Atum Sólido ao Natural Gomes da Costa 170g,12.99,NaN,Carrefour Bairro MorumbiJardim Londrina,São Paulo - SP,SP,-23.61172,-46.74075,Sudeste
